In [8]:
from llama_index.llms.ollama import Ollama
MODEL="llama3.2"
PROMPT= "Be a good boy"
DATA_PATH="./db.txt"

In [9]:
llm = Ollama(model=MODEL)

In [10]:
def retrieve_data(query):
    with open(DATA_PATH, 'r') as file:
        file_content = file.read()
    return file_content
    
# Step 3: Define the RAG pipeline with retrieval and generation
def rag_pipeline(query):
    # Retrieve relevant documents
    # retrieved_docs = [retrieve_data(query)]
    data = ""
    data = retrieve_data(query)
    # Build the prompt with the retrieved documents
    prompt = PROMPT + data + query
    # print(prompt)
    # Generate the response with Ollama
    response = llm.complete(prompt)
    return response  # Extracting the generated answer text



In [11]:
# Test the RAG pipeline
# query = "Give me insight about the main KPIs?"
query = "Give me insights about the important KPIs"

answer = rag_pipeline(query)
print(answer)

Based on the provided dataset, here are some insights about the important KPIs:

1. **Time-Related KPIs**:
	* `working_time`: This KPI indicates the total time spent in active operation for each machine. The mean working time for Large Capacity Cutting Machine 1 is around 4531 seconds, indicating that it spends most of its time in active operation.
	* `idle_time`: This KPI shows the total time spent when the machine is idle. For example, Riveting Machine has an average idle time of 10522 seconds, indicating that it often spends a significant amount of time not being used.
2. **Energy and Cost-Related KPIs**:
	* `consumption`: This KPI measures the total energy consumption for each machine. The mean consumption for Laser Cutter is around 1036.503 kWh, indicating that it consumes a significant amount of energy.
	* `cost`: This KPI shows the cost associated with energy consumption. For example, Riveting Machine has an average cost of 1029 euro per kWh, making it one of the most expensive 